In [4]:
# !pip install spacy

In [6]:
# !pip install sentence_transformers --index-url=https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple --trusted-host=artifactory.alight.com

In [8]:
# !pip install en_core_web_sm-3.1.0-py3-none-any.whl

In [ ]:
import spacy
import en_core_web_sm

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

nlp = spacy.load("en_core_web_sm")

In [11]:
df_combined_web_iva_search = pd.read_csv("s3://adl-core-sagemaker-studio/external/IVA/combined_new_adult-child_outer_new.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (0,1,2,4,5,9,10,11,12,13,14,15,16,17,18,20,21,23,24,25,26,27,28,29,31,33,39,41,56,57,58,59,63,64,65,67,68,70,71,74,75,77,78,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,106,108,109,110,111,112,113,114,115,116,117,118,119,122,123,124,125,126,127,128,129,130,131,132,134,135,136,137,138,155,156,159,160,161,164,165) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
df_combined_web_iva_search[['input','search_text','page_name']].head()

,input,search_text,page_name
0,step child a dependent?,NaN,NaN
1,how do i change my dependent daycare deduction,NaN,NaN
2,Child last name change,NaN,NaN
3,i am trying to add my children as beneficiarie...,NaN,NaN
4,how much timeoff do i get after the birth of m...,NaN,NaN


In [13]:
df_iva = pd.read_csv("s3://adl-core-sagemaker-studio/external/Deepali/IVA_cleaned_labelled(session_id_added).csv")

In [14]:
df_iva.columns

Index(['Unnamed: 0', 'entry_id', 'client_id', 'person_internal_id',
       'input_orig', 'labels', 'input_cleaned', 'input_cleaned_dl',
       'next_unit_hit', 'previous_unit_hit', 'unit_name', 'response_text',
       'session_id'],
      dtype='object')

In [15]:
df_iva1 = df_iva.drop(['Unnamed: 0','entry_id','client_id','person_internal_id','next_unit_hit',
             'previous_unit_hit','response_text','session_id'], axis=1)

In [16]:
df_iva1.head()

,input_orig,labels,input_cleaned,input_cleaned_dl,unit_name
0,Open enrollment,Enrollment,open enrol,open enrollment,Annual Enrollment Clarifier
1,enroll in hra,HRA,enrol hra,enroll in hra,Health Reimbursement Account Clarifier
2,TIRE DISCOUNT,Discounts Issue,tire discount,tire discount,Discounts Clarifier
3,eligible,HSA related,elig,eligible,Health Savings Account (HSA) Eligible Expenses
4,Need to update my mail address,General Acount issue,need updat mail address,need to update my mail address,Manage Address


In [17]:
df_iva2 = df_iva1.drop(['input_cleaned','input_cleaned_dl'], axis=1)

In [18]:
df_iva2.head()

,input_orig,labels,unit_name
0,Open enrollment,Enrollment,Annual Enrollment Clarifier
1,enroll in hra,HRA,Health Reimbursement Account Clarifier
2,TIRE DISCOUNT,Discounts Issue,Discounts Clarifier
3,eligible,HSA related,Health Savings Account (HSA) Eligible Expenses
4,Need to update my mail address,General Acount issue,Manage Address


In [19]:
df_concat = pd.concat([df_combined_web_iva_search, df_iva2], axis=1)

In [20]:
# df_concat.columns.to_list()

In [21]:
df_concat2 = df_concat[['input','input_orig','search_text','page_name','labels','session_start_cst']]

In [22]:
df_concat2.head()

,input,input_orig,search_text,page_name,labels,session_start_cst
0,step child a dependent?,Open enrollment,NaN,NaN,Enrollment,2017-12-05 21:48:56
1,how do i change my dependent daycare deduction,enroll in hra,NaN,NaN,HRA,2017-12-11 09:45:09
2,Child last name change,TIRE DISCOUNT,NaN,NaN,Discounts Issue,2018-01-22 08:12:28
3,i am trying to add my children as beneficiarie...,eligible,NaN,NaN,HSA related,2017-10-15 10:34:01
4,how much timeoff do i get after the birth of m...,Need to update my mail address,NaN,NaN,General Acount issue,2017-12-29 10:46:58


In [23]:
df_concat2.shape

(2330066, 6)

In [24]:
df_concat2.drop_duplicates(inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [25]:
df_concat2.shape

(1971001, 6)

In [26]:
import string
import nltk
nltk.data.path.append('../../nltk_data')
from nltk.corpus import stopwords
# nltk.download('stopwords')


In [27]:
def clean_text(df, text_cols):
    # Create a new dataframe to hold the cleaned text columns
    cleaned_df = pd.DataFrame()
    
    # Define the list of stopwords
    stop_words = set(stopwords.words('english'))
    
    # Clean each text column and add it to the cleaned dataframe
    for text_col in text_cols:
        text_list = df[text_col].tolist()
        text_list = [str(text) for text in text_list]
        text_list = [text for text in text_list if text.strip() and not
                     set(text).issubset(set(string.punctuation + string.whitespace))]
        text_list = [x.lower() for x in text_list]
        translator = str.maketrans(string.punctuation + string.digits + "_", " " * len(
            string.punctuation + string.digits + "_"))
        cleaned_list = []
        for text in text_list:
            cleaned_text = text.translate(translator)
            cleaned_text = ' '.join(cleaned_text.split())
            cleaned_text = ' '.join([word for word in cleaned_text.split() if word not in stop_words])
            cleaned_list.append(cleaned_text)
        cleaned_df[text_col] = cleaned_list
    
    # Add the non-text columns to the cleaned dataframe
    for col in df.columns:
        if col not in text_cols:
            cleaned_df[col] = df[col]
    
    return cleaned_df


In [28]:
df_concat3 = clean_text(df_concat2, text_cols=['input','search_text','page_name'])

In [29]:
df_concat3.head(5)

,input,search_text,page_name,input_orig,labels,session_start_cst
0,step child dependent,nan,nan,Open enrollment,Enrollment,2017-12-05 21:48:56
1,change dependent daycare deduction,nan,nan,enroll in hra,HRA,2017-12-11 09:45:09
2,child last name change,nan,nan,TIRE DISCOUNT,Discounts Issue,2018-01-22 08:12:28
3,trying add children beneficiaries life ins,nan,nan,eligible,HSA related,2017-10-15 10:34:01
4,much timeoff get birth child,nan,nan,Need to update my mail address,General Acount issue,2017-12-29 10:46:58


In [30]:
df_concat4 = df_concat3.replace('nan', '')
df_concat4.head()

,input,search_text,page_name,input_orig,labels,session_start_cst
0,step child dependent,,,Open enrollment,Enrollment,2017-12-05 21:48:56
1,change dependent daycare deduction,,,enroll in hra,HRA,2017-12-11 09:45:09
2,child last name change,,,TIRE DISCOUNT,Discounts Issue,2018-01-22 08:12:28
3,trying add children beneficiaries life ins,,,eligible,HSA related,2017-10-15 10:34:01
4,much timeoff get birth child,,,Need to update my mail address,General Acount issue,2017-12-29 10:46:58


In [31]:
df_concat4['text'] = df_concat4[['input', 'search_text', 'page_name']].apply(lambda x: ' '.join([str(i) for i in x if not pd.isna(i)]), axis=1)

In [32]:
df_concat4.drop(['input','search_text','page_name'], inplace=True, axis=1)

In [33]:
df_concat4.drop(['input_orig'], inplace=True, axis=1)

In [34]:
df_concat4.head()

,labels,session_start_cst,text
0,Enrollment,2017-12-05 21:48:56,step child dependent
1,HRA,2017-12-11 09:45:09,change dependent daycare deduction
2,Discounts Issue,2018-01-22 08:12:28,child last name change
3,HSA related,2017-10-15 10:34:01,trying add children beneficiaries life ins
4,General Acount issue,2017-12-29 10:46:58,much timeoff get birth child


In [35]:
df_concat4.drop_duplicates(subset=['text'], inplace=True)

In [36]:
df_concat4.shape

(111098, 3)

In [37]:
# Get the value counts of the 'labels' column
label_counts = df_concat4['labels'].value_counts()
print(sum(label_counts>500))
# Filter the dataframe to only include rows where the label count is greater than 10000
df_concat5 = df_concat4[df_concat4['labels'].isin(label_counts[label_counts > 10000].index)]

# Get the shape of the resulting filtered dataframe
df_concat5_shape = df_concat5.shape

41


In [38]:
df_concat5_shape

(36517, 3)

In [39]:
df_concat5['labels'].value_counts()

Other              25225
Health Benefits    11292
Name: labels, dtype: int64

In [40]:
df_concat4[df_concat4['text']=='nan '].value_counts()

Series([], dtype: int64)

In [41]:
df_concat4.to_excel('cc_df_concat4.xlsx')

### finding the texts which contain exact phrases from synonyms list

In [42]:
words_3 = ['day care', 'creche', 'childcare','daycare','nurseryschool','after school care',
           'pre school', 'child', 'baby', 'minor care','minorcare','offspring',
           'daynursery', 'infantschool','pre K', 'childcarer', 'childcarers','girl child',
 'infantcare', 'play school', 'playgroup','boy child','Adolescent','Little one','Young one',
 'nursery', 'nursery school', 'preschool', 'after schoolcare', 'day nursery', 'infant school', 
 'infant care', 'playschool', 'play group', 'kindergarten', 'childminding', 'babysitting',
           'babysitter', 
 'nanny care', 'children supervision', 'toddler care', 'prekindergarten',
           'Junior care','Sprout care',
'child minding', 'baby sitting', 'baby sitter', 'children',
 'nannycare', 'childrensupervision','child supervision', 'childsupervision', 
           'toddlercare','foster', 'childs','childrens',
           'stepchild','step daughter', 'step son', 'grandchildren','grandchild',
          'stepchildren','childbirth',"children's", 'childhood',
          'childsupport','childcarereimbursement','childern','childrren',
           'grndchild','childre','granchild','achild','stephchild','childen','childbrith',
           'childrem','childeren','childd','childrfen ','mychildren',
           'mychild','childres','childresn','childcareplus','childcarea','stepchildrens',
           'childbirthing','dependentchild','childer','childacre','childrene','childrent','childcar',
'childcarew','childcrare','childacre','grandchildern','grandchildreni','childten',
          'childrewn','ghandchild','childrne','childtren','childreren','insurancechildren',
           'mybchildren','childare','grandchilddren','childcard',
          'childcard','grandchilddren','childare','mybchildren','insurancechildren',
           'childfrens','childreren','childtren','childrne','ghandchild','childrewn','childm',
          'childplus','childiren','childred','childrena','childrenen','childredn','childrebn',
           'childred','childlren','childrins','childbonding','childn']
words_4 = list(set([word.lower() for word in words_3]))

len(words_4)


122

In [43]:
pd.set_option('display.max_colwidth', None)

In [44]:
mask = (df_concat4['text'].str.contains(r'\b(' + '|'.join(words_4) + r')\b', case=False, na=False))

df_concat4['category'] = ''
df_concat4.loc[mask, 'category'] = 'Child care'
# df_combined_web_iva_search.loc[df_combined_web_iva_search['category'] == '', 'category'] = 'Other'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [45]:
df_concat4.sample(20)

,labels,session_start_cst,text,category
400634,General Account issue,2023-01-23 09:32:00,need add child born insurance,Child care
60843,Smoking,2020-10-27 14:51:46,email send dependent daycare form contentpage child adult pet care resources,Child care
158288,Health Benefits,2022-09-26 11:57:28,uploaded document child care plus asking upload child care plus enroll hm cstm child care domain ben hm baseclientindicator client languageid en us,Child care
593632,Other,2022-05-11 08:50:12,child depenedtnt turns,Child care
3817,Payment Issue,2017-11-11 14:16:35,need change dependent child,Child care
192374,W-2 Form,2021-02-04 08:44:51,trying add new child insurance add social security number yet,Child care
350235,Other,2021-09-24 08:24:56,would ex wife able use health spending cards services covered children,Child care
778636,Health Benefits,NaN,tuition assistance children contentpage need find child care facility,Child care
1138602,NaN,NaN,onsite child care lcc,Child care
253581,Enrollment,2022-08-31 13:31:59,wanted know enroll started working deluxe completed benefits need add children thought able add,Child care


In [46]:
# df_concat4[df_concat4['category']=='Child care'].sample(20)

In [47]:
df_concat4[df_concat4['category']=='Child care'].shape

(109348, 4)

In [48]:
df_concat4.shape

(111098, 4)

### get text which are similar to phrases in synonnyms list for texts other than which are filtered above

In [76]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# def find_similar_sentences(df, sentences, synonyms_list, threshold=0.90, category_name='Child care'):

#     # Define an empty list to store the filtered rows
#     filtered_rows = []

#     for sentence in sentences:
#         # Tokenize the sentence
#         # tokens = tokenizer(sentence, padding=True, truncation=True, return_attention_mask=True,
#         #                    return_token_type_ids=False)
#         tokens = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
#         # print(tokens.keys())
#         # Get the BERT embeddings for the tokens
#         with torch.no_grad():
#             # outputs = model(input_ids=tokens['input_ids'], attention_mask=tokens['attention_mask'])
#             outputs = model(input_ids=torch.tensor(tokens['input_ids']), attention_mask=torch.tensor(tokens['attention_mask']))
#             embeddings = outputs.last_hidden_state.mean(dim=1)

#         # Define an empty list to store the matching tokens and phrases
#         matching_tokens_phrases = []

#         # Compute the cosine similarity between the sentence and each phrase/synonym
#         for phrase in synonyms_list:
#             # Tokenize the phrase
#             # phrase_tokens = tokenizer(phrase, padding=True, truncation=True, return_attention_mask=False,
#             #                           return_token_type_ids=False)
#             phrase_tokens = tokenizer(phrase, padding=True, truncation=True, return_tensors='pt')


#             # Get the BERT embeddings for the phrase
#             with torch.no_grad():
#                 phrase_outputs = model(**phrase_tokens)
#                 phrase_embeddings = phrase_outputs.last_hidden_state.mean(dim=1)

#             # Compute the cosine similarity between the sentence and the phrase
#             cos_sim = torch.nn.functional.cosine_similarity(embeddings, phrase_embeddings)

#             # Convert the tensor to a Python float using the .item() method
#             cos_sim = cos_sim.item()

#             if cos_sim > threshold:
        
#                 for i, token in enumerate(tokens['input_ids'][0]):
#                     # Get the token string and convert it from byte to string
#                     token_str = tokenizer.convert_ids_to_tokens(token.item())

#                     # Compute the cosine similarity between the token and the phrase
#                     token_cos_sim = torch.nn.functional.cosine_similarity(phrase_embeddings, embeddings[i].unsqueeze(0))
#                     token_cos_sim = token_cos_sim.item()

#                     # If the cosine similarity is above the threshold, add the matching token and phrase to the list
#                     if token_cos_sim > threshold:
#                         matching_tokens_phrases.append((token_str, phrase))
#                         break

#         # If there are any matching tokens and phrases, add the sentence to the filtered rows
#         if matching_tokens_phrases:
#             # filtered_rows.append((sentence, matching_tokens_phrases))
#             filtered_rows.append((sentence, tuple(matching_tokens_phrases)))


#     # Convert the list of similar sentences to a set to remove duplicates
#     filtered_rows = set(filtered_rows)

#     # Convert the filtered rows to a new dataframe
#     filtered_df = pd.DataFrame(columns=['text', 'matching_tokens_phrases'])
#     for row in filtered_rows:
#         filtered_df = filtered_df.append({'text': row[0], 'matching_tokens_phrases': row[1]}, ignore_index=True)

#     # Add a category column to the filtered dataframe
#     filtered_df['category'] = category_name

#     return filtered_df


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [50]:
# def find_similar_sentences(df, sentences, synonyms_list, threshold=0.90, category_name='Child care', batch_size=32):

#     filtered_rows = []  # create an empty list to hold the filtered rows
#     num_batches = int(np.ceil(len(sentences) / batch_size))  # compute the number of batches based on the batch size and number of sentences
    
#     # loop over each batch of sentences
#     for batch_idx in range(num_batches):
#         start_idx = batch_idx * batch_size
#         end_idx = min((batch_idx + 1) * batch_size, len(sentences))
#         batch_sentences = sentences[start_idx:end_idx]
#         # print(batch_sentences)
#         # Tokenize the batch of sentences using the BERT tokenizer
#         batch_tokens = tokenizer.batch_encode_plus(batch_sentences, padding=True, truncation=True, return_tensors='pt')

#         # Get the BERT embeddings for the tokens
#         with torch.no_grad():
#             batch_outputs = model(**batch_tokens)
#             batch_embeddings = batch_outputs.last_hidden_state.mean(dim=1)

#         # Iterate over each sentence in the batch
#         for sentence_idx in range(len(batch_sentences)):
#             # Get the tokens for the current sentence
#             sentence_tokens = tokenizer.convert_ids_to_tokens(batch_tokens['input_ids'][sentence_idx])

#             # Compute the cosine similarity between the batch embeddings and the phrase embeddings
#             cos_sim = torch.nn.functional.cosine_similarity(batch_embeddings[sentence_idx].unsqueeze(0), batch_embeddings, dim=1)

#             # Get the cosine similarity for the current sentence
#             sentence_cos_sim = cos_sim[0].item()

#             # If the cosine similarity is above the threshold, directly add the sentence to the filtered rows
#             if sentence_cos_sim >= threshold:
#                 filtered_rows.append((batch_sentences[sentence_idx], ()))
                
#             # If the cosine similarity is below the threshold, find the matching tokens and phrases
#             else:
#                 matching_tokens_phrases = []

#                 # Iterate over each token in the sentence
#                 for token_idx in range(len(sentence_tokens)):
#                     token_str = sentence_tokens[token_idx]

#                     # Compute the cosine similarity between the token and the phrases in the synonyms list
#                     for phrase in synonyms_list:
#                         # Tokenize the phrase using the BERT tokenizer
#                         phrase_tokens = tokenizer(phrase, padding=True, truncation=True, return_tensors='pt')

#                         # Get the BERT embeddings for the phrase
#                         with torch.no_grad():
#                             phrase_outputs = model(**phrase_tokens)
#                             phrase_embeddings = phrase_outputs.last_hidden_state.mean(dim=1)

#                         # Compute the cosine similarity between the token and the phrase
#                         token_cos_sim = torch.nn.functional.cosine_similarity(phrase_embeddings, batch_embeddings[sentence_idx][token_idx].unsqueeze(0))
#                         token_cos_sim = token_cos_sim.item()

#                         # If the cosine similarity is above the threshold, add the matching token and phrase to the list
#                         if token_cos_sim > threshold:
#                             matching_tokens_phrases.append((token_str, phrase))

#                 # If there are any matching tokens and phrases, add the sentence to the filtered rows
#                 if matching_tokens_phrases:
#                     filtered_rows.append((batch_sentences[sentence_idx], tuple(matching_tokens_phrases)))

#     # Convert the list of similar sentences to a set to remove duplicates
#     filtered_rows = set(filtered_rows)
#     # print(filtered_rows)
#     # Convert the filtered rows to a new dataframe
#     filtered_df = pd.DataFrame(columns=['text', 'matching_tokens_phrases', 'category'])
#     for row in filtered_rows:
#         filtered_df = filtered_df.append({'text': row[0], 'matching_tokens_phrases': row[1], 'category': category_name}, ignore_index=True)
    
#     return filtered_df

In [51]:
import numpy as np
import torch
import pandas as pd
from torch.nn.parallel import DataParallel
from transformers import AutoTokenizer, AutoModel
import os

def find_similar_sentences(df, sentences, synonyms_list, threshold=0.90, category_name='Child care',
                           batch_size=52):
    # Create a tokenizer and a pre-trained BERT model
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    model = AutoModel.from_pretrained('bert-base-uncased')
    
    # Use GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Create a set of unique synonyms
    unique_synonyms = set(synonyms_list)

    # Compute the average BERT embeddings for each unique phrase in the synonym list
    phrase_embeddings = []
    for phrase in unique_synonyms:
        phrase_tokens = tokenizer.encode(phrase, add_special_tokens=False, truncation=True)
        with torch.no_grad():
            phrase_outputs = model(torch.tensor(phrase_tokens).unsqueeze(0).to(device))
            phrase_embedding = phrase_outputs.last_hidden_state.mean(dim=1)
            phrase_embeddings.append(phrase_embedding)
    phrase_embeddings = torch.cat(phrase_embeddings, dim=0)

    # Use DataParallel to parallelize computations across multiple GPUs
    print(torch.cuda.device_count())   
    if torch.cuda.device_count() > 1:
        model = DataParallel(model)

    # Create an empty list to hold the filtered rows
    filtered_rows = []

    # Loop over each batch of sentences
    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i:i + batch_size]

        # Tokenize the batch of sentences using the BERT tokenizer
        batch_tokens = tokenizer.batch_encode_plus(batch_sentences, padding=True, truncation=True, return_tensors='pt').to(device)

        # Get the BERT embeddings for the tokens
        with torch.no_grad():
            batch_outputs = model(**batch_tokens)
            batch_embeddings = batch_outputs.last_hidden_state.mean(dim=1)

        # Compute the cosine similarity between the batch embeddings and the phrase embeddings
        cos_sim = torch.nn.functional.cosine_similarity(batch_embeddings, phrase_embeddings, dim=1)

        # Get the indices of the sentences that are above the threshold
        sim_indices = torch.where(cos_sim >= threshold)[0].tolist()

        # Iterate over each similar sentence
        for idx in sim_indices:
            filtered_rows.append((batch_sentences[idx], ()))
        
        matching_tokens_phrases = []
        # Iterate over each sentence in the batch
        for j in range(len(batch_sentences)):
            # If the sentence has already been added to the filtered rows, skip it
            if j in sim_indices:
                continue

            # matching_tokens_phrases = []

            # Iterate over each token in the sentence
            # for token_idx, token in enumerate(batch_tokens['input_ids'][j].numpy())
            for token_idx, token in enumerate(batch_tokens['input_ids'][j].cpu().numpy()):
                token_scalar = int(token)  # Convert numpy array to scalar
                token_str = tokenizer.convert_ids_to_tokens(token_scalar)

                # Compute the cosine similarity between the token and the phrases in the synonyms list
                for phrase in unique_synonyms:
                    # Tokenize the phrase using the BERT tokenizer
                    phrase_tokens = tokenizer.encode(phrase, add_special_tokens=False, truncation=True)

                    # Get the BERT embeddings for the phrase
                    with torch.no_grad():
                        phrase_outputs = model(torch.tensor(phrase_tokens).unsqueeze(0))
                        phrase_embedding = phrase_outputs.last_hidden_state.mean(dim=1)

                    # Compute the cosine similarity between the token and the phrase
                    token_cos_sim = torch.nn.functional.cosine_similarity(phrase_embedding, batch_embeddings[j][token_idx].unsqueeze(0))
                    token_cos_sim = token_cos_sim.item()

                    # If the cosine similarity is above the threshold, add the matching token and phrase to the list
                    if token_cos_sim > threshold:
                        matching_tokens_phrases.append((token_str, phrase))

                        
            # If there are any matching tokens and phrases, add the sentence to the filtered rows
            if matching_tokens_phrases:
                filtered_rows.append((batch_sentences[j], tuple(matching_tokens_phrases)))
                
    # Convert the list of similar sentences to a set to remove duplicates
    filtered_rows = set(filtered_rows)
    # print(filtered_rows)
    # Convert the filtered rows to a new dataframe
    filtered_df = pd.DataFrame(columns=['text', 'matching_tokens_phrases', 'category'])
    for row in filtered_rows:
        filtered_df = filtered_df.append({'text': row[0], 'matching_tokens_phrases': row[1], 'category': category_name}, ignore_index=True)
    
    return filtered_df

In [52]:
df_concat4 = df_concat4.dropna(subset=['text'])

In [53]:
similar_df2 = find_similar_sentences(df_concat4, 
                                    df_concat4[df_concat4['category']=='']['text'].head(100).to_list(), 
                                    words_4)
# similar_df2 = find_similar_sentences(df_concat4, ['elder care','older care'], words_4)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0


RuntimeError: The size of tensor a (52) must match the size of tensor b (122) at non-singleton dimension 0

In [80]:
similar_df2

,text,matching_tokens_phrases,category
0,eldercare benefits,(),Child care
1,need information elder care fsa covers husband much older need assistance work hours,(),Child care
2,user friendly everything referred back computer elderly retired employees like computers don’t remember password logged remember anyway need know p,(),Child care
3,husband hospitalized may need home physical therapy home care released trying figure covered looks like,(),Child care


In [180]:
only_CC_df = pd.concat([df_concat4[df_concat4['category']=='Child care'], 
                       similar_df2]).sample(frac=1)

In [181]:
only_CC_df.shape

(109364, 5)

In [182]:
only_CC_df.drop_duplicates(inplace=True)

In [183]:
only_CC_df.shape

(109364, 5)

In [184]:
non_similar_df = df_concat4[~df_concat4['text'].isin(only_CC_df['text'])]

non_similar_df['category'] = 'Other'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [185]:
non_similar_df.shape

(1734, 4)

In [186]:
df_concatenated = pd.concat([only_CC_df, non_similar_df]).sample(frac=1).reset_index(drop=True)

In [187]:
pd.set_option('display.max_colwidth', None)

In [188]:
df_concatenated.shape

(111098, 5)

In [189]:
df_concatenated[(df_concatenated.text.str.contains('child')) & (df_concatenated.category=='Other')].sample(20)

,labels,session_start_cst,text,category,matching_tokens_phrases
39388,HRA,2023-01-27 11:51:19,change childerns last name,Other,NaN
30476,Coverage Issue,2023-01-09 14:14:19,added childand plan came,Other,NaN
45861,Loan related,2023-03-15 09:52:38,need know difference price family vs chchild,Other,NaN
46780,Health Benefits,2023-01-05 11:13:37,add vchildern isurance,Other,NaN
71296,Other,2021-02-08 09:45:26,cover gradnchild depend care,Other,NaN
105914,General Account issue,2020-10-30 15:35:14,covered childe life insurance,Other,NaN
107378,view/print,2017-12-06 09:31:35,son receiving michild government program longer covered program income gone need add insurance process need,Other,NaN
51685,Claims,NaN,extended childca,Other,NaN
9012,NaN,NaN,rothchild,Other,NaN
37126,Form 1095 Issue,2022-09-16 11:50:41,benefits savings retirement change made change transaction qsc actsp childgaineligelsewhe,Other,NaN


In [190]:
df_concatenated[(df_concatenated.text.str.contains('child')) & (df_concatenated.category=='Other')].shape

(334, 5)

In [191]:
df_concatenated.to_excel('BERT_CC_other_data_90%_sim.xlsx')

In [6]:
import pandas as pd
df_concatenated = pd.read_excel('BERT_CC_other_data_90%_sim.xlsx')

In [7]:
pd.reset_option('display.max_colwidth')

In [8]:
df_concatenated[df_concatenated.category=='Child care'].to_excel('BERT_only_child_care_training_data_90%_sim.xlsx')

In [9]:
## number og labels in label col where value of category col is Other
df_concatenated[df_concatenated['category'] == 'Other']['labels'].nunique()

113

In [10]:
df_concatenated2 = df_concatenated.drop('session_start_cst', axis=1)

In [11]:
df_concatenated2['labels'].isnull().sum()

1333

In [12]:
df_concatenated2['category'].isnull().sum()

0

In [13]:
df_concatenated2['text'].isnull().sum()

0

In [14]:
df_concatenated2['labels'] = df_concatenated2['labels'].fillna('empty')

In [19]:
df_concatenated2.category.value_counts(normalize=True)

Child care    0.984392
Other         0.015608
Name: category, dtype: float64

In [20]:
df_concatenated2.shape

(111098, 5)

In [21]:
df_concatenated2.columns

Index(['Unnamed: 0', 'labels', 'text', 'category', 'matching_tokens_phrases'], dtype='object')

In [22]:
df_concatenated2.head()

,Unnamed: 0,labels,text,category,matching_tokens_phrases
0,0,Natural Disaster,recently child send correct link add insurance,Child care,NaN
1,1,Other,need help apllying back child care subsidy,Child care,NaN
2,2,Dependent Care Spending,submit documents child care plus hmcstmchildc...,Child care,NaN
3,3,Payroll Issue,go drop coverage adult child,Child care,NaN
4,4,Dependent issue,hello order medical card children,Child care,NaN


### text preprocess

In [35]:
import re
import numpy as np

# def count_intnt_entits(text):
#     doc = nlp(text)
#     intents = [token.text for token in doc if token.pos_ == 'VERB']
#     entities = [token.text for token in doc if token.pos_ in {'NOUN', 'PROPN', 'ADJ', 'NUM', 'ADV'}]
#     return len(intents), len(entities)

def count_intnt_entits(text):
    if str(text).isnumeric():
        return 0,0    
    try:
        doc = nlp(str(text))
        intents = [token.text for token in doc if token.pos_ == 'VERB']
        entities = [token.text for token in doc if token.pos_ in {'NOUN', 'PROPN', 'ADJ', 'NUM', 'ADV'}]
    except:
        print(text)
        raise
    return len(intents), len(entities)

def extract_ner_entities(sentence):
    doc = nlp(sentence)
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    return entities

def length_entities(list_entities):
    if (list_entities==np.nan or list_entities==None or list_entities==''):
        return 0
    else:
        return len(list_entities)
    
def filter_named_entities(text):
    # Process the text using Spacy
    doc = nlp(text)
    # Filter out named entities (ORG, PERSON, and GPE tags)
    filtered_words = [token.text for token in doc if token.ent_type_ not in ['ORG', 'PERSON', 'GPE', "LOC", "FAC"]]
    # Join the filtered words back into a string
    filtered_text = ' '.join(filtered_words)
    return filtered_text

In [36]:
import re

# def text_preprocess(dataframe, text_col, category_col):
#     # Drop duplicates and remove digits from the text column
#     # dataframe = dataframe[[text_col, category_col]].drop_duplicates()
#     # dataframe = dataframe[text_col].drop_duplicates()
#     print(type(text_col))
#     dataframe[text_col] = dataframe[text_col].str.replace('\d+', '')
    
#     # Count intents and entities and add new columns
#     dataframe[['no_of_intents', 'no_of_entities']] = dataframe.apply(lambda x: pd.Series(count_intnt_entits(x[text_col])), axis=1)  
    
#     # Extract named entities and add new columns
#     dataframe['ner_enities'] = ''
#     dataframe.loc[dataframe[text_col]!='', 'ner_enities'] = dataframe.loc[dataframe[text_col]!='', text_col].apply(extract_ner_entities)
#     dataframe['len_ner_enities'] = dataframe['ner_enities'].apply(length_entities)
#     dataframe3 = dataframe[dataframe['len_ner_enities']>0]
#     dataframe3[text_col] = dataframe3[text_col].apply(filter_named_entities)
#     dataframe6 = pd.concat([dataframe[dataframe['len_ner_enities']==0], dataframe3], axis = 0)
#     dataframe6 = dataframe6.drop(['no_of_intents','no_of_entities','ner_enities','len_ner_enities'], axis=1)

#     dataframe6[text_col] = dataframe6[text_col].str.strip()
    
#     return dataframe6

def text_preprocess(dataframe, text_col, category_col):
    # Drop duplicates and remove digits from the text column
    dataframe = dataframe[[text_col, category_col]].drop_duplicates()
    dataframe[text_col] = dataframe[text_col].str.replace('\d+', '')
    
    # Count intents and entities and add new columns
    dataframe[['no_of_intents', 'no_of_entities']] = dataframe.apply(lambda x: pd.Series(count_intnt_entits(x[text_col])), axis=1)  
    
    # Extract named entities and add new columns
    dataframe['ner_enities'] = ''
    dataframe.loc[dataframe[text_col]!='', 'ner_enities'] = dataframe.loc[dataframe[text_col]!='', text_col].apply(extract_ner_entities)
    dataframe['len_ner_enities'] = dataframe['ner_enities'].apply(length_entities)
    dataframe3 = dataframe[dataframe['len_ner_enities']>0]
    dataframe3[text_col] = dataframe3[text_col].apply(filter_named_entities)
    dataframe6 = pd.concat([dataframe[dataframe['len_ner_enities']==0], dataframe3], axis = 0)
    dataframe6 = dataframe6.drop(['no_of_intents','no_of_entities','ner_enities','len_ner_enities'], axis=1)

    dataframe6[text_col] = dataframe6[text_col].str.strip()
    
    return dataframe6


In [37]:
df_concatenated2.columns

Index(['Unnamed: 0', 'labels', 'text', 'category', 'matching_tokens_phrases'], dtype='object')

In [38]:
df_combined_cc_model_data_2 = text_preprocess(df_concatenated2, 'text','category')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [39]:
df_combined_cc_model_data_2.shape

(111098, 2)

In [40]:
df_combined_cc_model_data_2.columns

Index(['text', 'category'], dtype='object')

In [41]:
df_combined_cc_model_data_2['category'].value_counts()

Child care    109364
Other           1734
Name: category, dtype: int64

In [42]:
df_combined_cc_model_data_2.to_excel('final_cc_model_data_v6.xlsx')

In [23]:
import pandas as pd
df_combined_cc_model_data_2 = pd.read_excel('final_cc_model_data_v6.xlsx')

In [24]:
df_combined_cc_model_data_2.isnull().sum()

Unnamed: 0      0
text          286
category        0
dtype: int64

In [25]:
df_combined_cc_model_data_2 = df_combined_cc_model_data_2.dropna(subset=['text'])

In [36]:
data = df_combined_cc_model_data_2[['text', 'category']].rename(columns={'category':'label'})
dataset = list(data.itertuples(index=False, name=None))

# dataset[:1]

In [31]:
# import torch
# import torch.nn as nn
# from transformers import BertTokenizer, BertModel
# from torch.nn.utils.rnn import pad_sequence

# class TextClassifier(nn.Module):
#     def __init__(self, num_labels):
#         super(TextClassifier, self).__init__()
#         self.bert = BertModel.from_pretrained('bert-base-uncased')
#         self.dropout = nn.Dropout(0.1)
#         self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

#     def forward(self, input_ids, attention_mask):
#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#         pooled_output = outputs.pooler_output
#         pooled_output = self.dropout(pooled_output)
#         logits = self.classifier(pooled_output)
#         return logits

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Tokenize the dataset
# tokenized_dataset = []
# for text, label in dataset:
#     input_ids = tokenizer.encode(text, add_special_tokens=True)
#     attention_mask = [1] * len(input_ids)
#     tokenized_dataset.append((input_ids, attention_mask, label))

# # Convert the tokenized dataset into PyTorch tensors
# input_ids = pad_sequence([torch.tensor(x[0]) for x in tokenized_dataset], batch_first=True)
# attention_mask = pad_sequence([torch.tensor(x[1]) for x in tokenized_dataset], batch_first=True)
# labels = torch.tensor([1 if x[2] == "Elder care" else 0 for x in tokenized_dataset])

# # Define the training parameters
# batch_size = 2
# num_epochs = 10
# learning_rate = 1e-5

# # Create the DataLoader
# dataset = torch.utils.data.TensorDataset(input_ids, attention_mask, labels)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Define the model, optimizer, and loss function
# model = TextClassifier(num_labels=2)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# loss_fn = nn.CrossEntropyLoss()

# # Train the model
# for epoch in range(num_epochs):
#     for batch in dataloader:
#         input_ids, attention_mask, labels = batch
#         optimizer.zero_grad()
#         logits = model(input_ids, attention_mask)
#         loss = loss_fn(logits, labels)
#         loss.backward()
#         optimizer.step()
#     print("Epoch {}/{} complete. Loss: {}".format(epoch+1, num_epochs, loss.item()))

In [35]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torch.nn.utils.rnn import pad_sequence

class TextClassifier(nn.Module):
    def __init__(self, num_labels):
        super(TextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
tokenized_dataset = []
for text, label in dataset:
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    attention_mask = [1] * len(input_ids)
    tokenized_dataset.append((input_ids, attention_mask, label))

# Convert the tokenized dataset into PyTorch tensors
input_ids = pad_sequence([torch.tensor(x[0]) for x in tokenized_dataset], batch_first=True)
attention_mask = pad_sequence([torch.tensor(x[1]) for x in tokenized_dataset], batch_first=True)
labels = torch.tensor([1 if x[2] == "Child care" else 0 for x in tokenized_dataset])

# Define the training parameters
batch_size = 128
num_epochs = 10
learning_rate = 1e-5
checkpoint_interval = 1 # save a checkpoint every 2 epochs

# Create the DataLoader
dataset = torch.utils.data.TensorDataset(input_ids, attention_mask, labels)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the model, optimizer, and loss function
model = TextClassifier(num_labels=2)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

# Train the model
for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

    print("Epoch {}/{} complete. Loss: {}".format(epoch+1, num_epochs, loss.item()))

    # Save a checkpoint every checkpoint_interval epochs
    if (epoch+1) % checkpoint_interval == 0:
        checkpoint_path = f"bert_cc_model_checkpoint_{epoch+1}.pt"
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, checkpoint_path)

ValueError: too many values to unpack (expected 2)

In [37]:
import os
# Resume training from the latest checkpoint
latest_checkpoint = torch.load(os.path.join(checkpoint_dir, 'bert_cc_latest_checkpoint.pth'))
model.load_state_dict(latest_checkpoint['model_state_dict'])
optimizer.load_state_dict(latest_checkpoint['optimizer_state_dict'])
epoch = latest_checkpoint['epoch'] + 1
loss = latest_checkpoint['loss']

# Train the model
for epoch in range(epoch, num_epochs):
    for batch in dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
    print("Epoch {}/{} complete. Loss: {}".format(epoch+1, num_epochs, loss.item()))

    # Save a checkpoint after each epoch
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'loss': loss
    }
    torch.save(checkpoint, os.path.join(checkpoint_dir, 'epoch_{}.pth'.format(epoch)))
    torch.save(checkpoint, os.path.join(checkpoint_dir, 'bert_cc_latest_checkpoint.pth'))

NameError: name 'checkpoint_dir' is not defined

In [20]:
# Save the model
torch.save(model.state_dict(), "Bert_EC_model_v2/ec_model_10_epoch_v2.pth")

In [20]:
# # Use the trained model to make predictions
# input_text = "My grandmother needs elder care services."
# input_ids = torch.tensor([tokenizer.encode(input_text, add_special_tokens=True)])
# attention_mask = torch.tensor([[int(token_id > 0) for token_id in input_ids[0]]])
# logits = model(input_ids, attention_mask)
# probs = nn.functional.softmax(logits, dim=-1)
# predicted_label = torch.argmax(probs, dim=-1)

# # Print the predicted label
# if predicted_label == 1:
#     print("Elder care")
# else:
#     print("Other")

Elder care


In [33]:
def count_intnt_entits(text):
    if str(text).isnumeric():
        return 0,0    
    try:
        doc = nlp(str(text))
        intents = [token.text for token in doc if token.pos_ == 'VERB']
        entities = [token.text for token in doc if token.pos_ in {'NOUN', 'PROPN', 'ADJ', 'NUM', 'ADV'}]
    except:
        print(text)
        raise
    return len(intents), len(entities)

def extract_ner_entities(sentence):
    doc = nlp(str(sentence))
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    return entities

def length_entities(list_entities):
    if (list_entities==np.nan or list_entities==None or list_entities==''):
        return 0
    else:
        return len(list_entities)
    
def filter_named_entities(text):
    # Process the text using Spacy
    doc = nlp(str(text))
    # Filter out named entities (ORG, PERSON, and GPE tags)
    filtered_words = [token.text for token in doc if token.ent_type_ not in ['ORG', 'PERSON', 'GPE', "LOC", "FAC"]]
    # Join the filtered words back into a string
    filtered_text = ' '.join(filtered_words)
    return filtered_text

def text_preprocess(col):
    df = pd.DataFrame({ 'text': col })
    df = df.drop_duplicates()
    df['text'] = df['text'].str.replace('\d+', '')
    df[['no_of_intents', 'no_of_entities']] = df.apply(lambda x: pd.Series(count_intnt_entits(x['text'])), axis=1)  

    df['ner_enities'] = ''
    df.loc[df['text']!='', 'ner_enities'] = df.loc[df['text']!='', 'text'].apply(extract_ner_entities)
    df['len_ner_enities'] = df['ner_enities'].apply(length_entities)
    df3 = df[df['len_ner_enities']>0]
    df3['text'] = df3['text'].apply(filter_named_entities)
    df6 = pd.concat([df[df['len_ner_enities']==0], df3], axis = 0)
    df6 = df6.drop(['no_of_intents','no_of_entities','ner_enities','len_ner_enities'], axis=1)

    df6['text'] = df6['text'].str.strip()
    
    return df6['text'].to_list()

def clean_text(text_list):
    # Clean the text
    text_list = text_preprocess(text_list)
    #text_list = [text for text in text_list if text.strip() and not set(text).issubset(set(string.punctuation + string.whitespace))]
    text_list1 = []
    for text in text_list:
        if isinstance(text, str):
            if text.strip() and not set(text).issubset(set(string.punctuation + string.whitespace)):
                text_list1.append(text)
            
    text_list = text_list1
    
    text_list = [x.lower() for x in text_list]
    # Define a translation table to replace punctuation and special characters with empty string
    translator = str.maketrans(string.punctuation + "_", " " * len(string.punctuation + "_"))
    # Loop through each text in the list and clean it
    cleaned_list = []
    for text in text_list:
        # Replace punctuation and special characters with empty string
        cleaned_text = text.translate(translator)
        # Remove any remaining special characters, punctuation, or whitespaces
        cleaned_text = ' '.join(cleaned_text.split())
        cleaned_list.append(cleaned_text)
    
    return cleaned_list

In [78]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Load the saved model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.load_state_dict(torch.load("Bert_EC_model/ec_model_10_epoch.pth"))

# Load the input Excel file
df = pd.read_excel('unseen_data.xlsx')
cleaned_text_list = clean_text(df['text'].to_list())
# Make predictions for each text in the Excel file
predictions = []
for text in cleaned_text_list:
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    attention_mask = torch.tensor([[int(token_id > 0) for token_id in input_ids[0]]])
    logits = model(input_ids, attention_mask).logits
    probs = nn.functional.softmax(logits, dim=-1)
    predicted_label = torch.argmax(probs, dim=-1)
    if predicted_label == 1:
        predictions.append((text, 'Elder care', probs[0][1].item()))
    else:
        predictions.append((text, 'Other', probs[0][0].item()))


# Save the predictions to a new file
df_pred = pd.DataFrame(predictions, columns=['text', 'prediction', 'probability'])
df_pred.to_excel('bert_ec_pred.xlsx', index=False)

In [81]:
list_of_texts = ["senior-citizen?care expense reimbursement", 
                 "elderly*care plus",
                 "aging care home", "retirees care reimbirsement", 
                 "senior assistance required","Daycare@expense reimbursement",
                 "baby care licensed", "oldsters care home","geriatric care home",
                 "contentPage 2023 Eldercare!!!!!!!!!!!!!****@_Subsidy","Elder statesmen care",
                 "Elder women care","Silver generation care",
                 "contentPage {}[]/\|?><,.;:!@#+\t\n\r\f\v 2023 Elder care Subsidy","gerontology care",
                 "Elderly Care Plus Information"]

import torch.nn.functional as F

cleaned_text_list = clean_text(list_of_texts)

predictions = []
for text in cleaned_text_list:
    # Skip empty input strings
    if not text:
        continue
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    attention_mask = torch.tensor([[int(token_id > 0) for token_id in input_ids[0]]])
    token_type_ids = torch.tensor([[0] * len(input_ids[0])]) # all tokens belong to the same segment in our case
    logits = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).logits

    probs = F.softmax(logits, dim=-1)
    predicted_label = torch.argmax(probs, dim=-1)
    if predicted_label == 1:
        predictions.append((text, 'Elder care', probs[0][1].item()))
    else:
        predictions.append((text, 'Other', probs[0][0].item()))

df_pred = pd.DataFrame(predictions, columns=['text', 'prediction', 'probability'])
df_pred


/tmp/ipykernel_74027/3565708624.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\d+', '')
/tmp/ipykernel_74027/3565708624.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['text'] = df3['text'].apply(filter_named_entities)


,text,prediction,probability
0,senior citizen care expense reimbursement,Elder care,0.999973
1,elderly care plus,Elder care,0.999985
2,aging care home,Elder care,0.999983
3,retirees care reimbirsement,Elder care,0.999841
4,senior assistance required,Elder care,0.999977
5,daycare expense reimbursement,Other,0.999982
6,baby care licensed,Other,0.999976
7,oldsters care home,Other,0.991685
8,contentpage eldercare subsidy,Elder care,0.999982
9,elder statesmen care,Elder care,0.999982


In [68]:
df.head()

,Unnamed: 0,text,category
0,0,i need a phone number for alight,Other
1,1,could pension plan selection be changed from j...,Other
2,2,where do i add my bank deposit info,Other
3,3,virgin pluse,Other
4,4,why att sent me enroll sheet,Other


In [70]:
# Load the saved model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.load_state_dict(torch.load("Bert_EC_model/ec_model_10_epoch.pth"))

# Load the input Excel file
df = pd.read_excel('labelled_unseen_data.xlsx')
cleaned_text_list = df['text'].to_list()
category_list = df['category'].to_list()

# Make predictions for each text in the Excel file
predictions = []
for i in range(len(cleaned_text_list)):
    text = cleaned_text_list[i]
    category = category_list[i]
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    attention_mask = torch.tensor([[int(token_id > 0) for token_id in input_ids[0]]])
    logits = model(input_ids, attention_mask).logits
    probs = nn.functional.softmax(logits, dim=-1)
    predicted_label = torch.argmax(probs, dim=-1)
    if predicted_label == 1:
        predictions.append((text, 'Elder care', category, probs[0][1].item()))
    else:
        predictions.append((text, 'Other', category, probs[0][0].item()))

# Save the predictions to a new file
df_pred = pd.DataFrame(predictions, columns=['text', 'prediction', 'category', 'probability'])
df_pred.to_excel('bert_labelled_ec_pred.xlsx', index=False)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

AttributeError: 'SequenceClassifierOutput' object has no attribute 'softmax'

### convert saved model .bin and config.json to .pkl compatible files

In [ ]:
# convert saved model .bin and config.json to .pkl compatible files
from transformers import AutoConfig, AutoModelForSequenceClassification
import torch
import pickle

# Load the model configuration from the config.json file
config = AutoConfig.from_pretrained('EC_model_outer_combined_texts_data_v6', num_labels=2)

# Load the model from the binary file using the configuration
model = AutoModelForSequenceClassification.from_pretrained('EC_model_outer_combined_texts_data_v6', config=config)

# Save the model and configuration as a pickle file
with open('EC_model_outer_combined_texts_data_v6/EC_model.pkl', 'wb') as f:
    pickle.dump((config, model.state_dict()), f)

## tagging the unseen_data for analysisng the predictions

### finding the texts which contain exact phrases from synonyms list

In [41]:
words_3 = ['older parent','older people','grand parents','elder','old parents','elder women',
 'silver generation','aged people', 'older women','older men','old age home','elder',
 'aged',
 'elderly people',
 'senior assistance',
 'aging-in-place',
 'aged population',
 'golden agers',
 'aging in place',
 'grey generation',
 'silver generation',
 'senior health',
 'aged population',
 'elderly companion',
 'golden agers',
 'senior citizen',
 'elder support',
 'elderly',
 'senior members',
 'elder population',
 'elderly residents',
 'senior assistance',
 'oldsters',
 'grey generation',
 'aging population',
 'elder statesmen',
 'elderly',
 'elderly people',
 'aging',
 'elderly residents',
 'elder',
 'elder women',
 'senior',
 'elder generation',
 'gerontology',
 'elderly population',
 'senior members',
 'retirees',
 'elderly population',
 'eldercare',
 'geriatric',
 'elder statesmen',
 'age related',
 'retirees',
 'third age population',
 'aging population',
 'elder population',
 'oldsters',
 'third age population','eldercae', 'eldercarr', 'eldermann', 
'aged home','eldercre','eldery','elderman','elders','eldercrae',]
words_4 = list(set([word.lower() for word in words_3]))
len(words_4)

49

In [42]:
df_cleaned = pd.DataFrame()
df_cleaned['text'] =cleaned_text_list
df_cleaned.head(5)

,text
0,welcomeuserfollowup
1,medical plan credit
2,welcomeuser
3,general purpose loans
4,chiropractor visits


In [43]:
mask = (df_cleaned['text'].str.contains(r'\b(' + '|'.join(words_4) + r')\b', case=False, na=False))

df_cleaned['category'] = ''
df_cleaned.loc[mask, 'category'] = 'Elder care'
# df_combined_web_iva_search.loc[df_combined_web_iva_search['category'] == '', 'category'] = 'Other'

df_cleaned.head(5)

/tmp/ipykernel_74027/3427755096.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = (df_cleaned['text'].str.contains(r'\b(' + '|'.join(words_4) + r')\b', case=False, na=False))


,text,category
0,welcomeuserfollowup,
1,medical plan credit,
2,welcomeuser,
3,general purpose loans,
4,chiropractor visits,


In [45]:
df_cleaned[df_cleaned['category']=='Elder care'].sample(1)

,text,category
5666,ongoing elderly care,Elder care


In [46]:
df_cleaned[df_cleaned['category']=='Elder care'].shape

(13, 2)

### get text which are similar to phrases in synonnyms list for texts other than which are filtered above

## labelling unseen data for matrix after prediction

In [50]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [51]:
def find_similar_sentences(df, sentences, phrases, threshold=0.95, category_name = 'Elder care'):
    # encode the phrases using the model
    phrase_embeddings = model.encode(phrases, convert_to_tensor=True)
    
    # initialize an empty list to store the similar sentences
    similar_sentences = []
    
    # iterate over the sentences
    for sentence in sentences:
        # encode the sentence using the model
        sentence_embedding = model.encode(sentence, convert_to_tensor=True)
        # reshape the sentence embedding to a 2D array
        sentence_embedding = sentence_embedding.reshape(1, -1)
        
        # calculate the cosine similarity between the sentence embedding and each phrase embedding
        cosine_scores = 1 - cosine_distances(sentence_embedding, phrase_embeddings)
        
        # convert the cosine similarity scores to a list
        scores_list = cosine_scores.tolist()[0]
        
        # iterate over the phrases and similarity scores and append the sentence to the list if it meets the threshold for at least one phrase
        for phrase, score in zip(phrases, scores_list):
            if score >= threshold:
                similar_sentences.append(sentence)
                break
    
    # convert the list of similar sentences to a set to remove duplicates
    similar_sentences = set(similar_sentences)
    
    # create a new dataframe containing only the rows with text that is in the set of similar sentences
    similar_df = df[df['text'].isin(similar_sentences)]
    similar_df['category']=category_name
    return similar_df

In [52]:
similar_df_unseen = find_similar_sentences(df_cleaned, 
                                    df_cleaned[df_cleaned['category']=='']['text'].to_list(), 
                                    words_4)

/tmp/ipykernel_74027/2749835889.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_df['category']=category_name


In [53]:
similar_df_unseen

,text,category
7095,retiremen,Elder care


In [54]:
unseen_EC_df = pd.concat([df_cleaned[df_cleaned['category']=='Elder care'], 
                       similar_df_unseen]).sample(frac=1)

In [55]:
unseen_EC_df.shape

(14, 2)

In [56]:
unseen_EC_df.drop_duplicates(inplace=True)

In [57]:
unseen_EC_df.shape

(14, 2)

In [58]:
# filter out the rows with similar text from the original DataFrame
non_similar_unseen_df = df_cleaned[~df_cleaned['text'].isin(unseen_EC_df['text'])]

# sample twice as many rows from the non-similar DataFrame as there are in the similar DataFrame
# non_similar_df = non_similar_df.sample(n=only_EC_df.shape[0]*2)
non_similar_unseen_df['category'] = 'Other'

/tmp/ipykernel_74027/3802977147.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_similar_unseen_df['category'] = 'Other'


In [59]:
df_unseen_concatenated = pd.concat([unseen_EC_df, non_similar_unseen_df]).sample(frac=1).reset_index(drop=True)
# df_concatenated = df_concatenated
# df_concatenated.drop(columns=['input', 'search_text', 'page_name'], inplace=True)

In [60]:
pd.set_option('display.max_colwidth', None)

In [61]:
df_unseen_concatenated.shape

(12584, 2)

In [62]:
df_unseen_concatenated[(df_unseen_concatenated.text.str.contains('elder')) & (df_unseen_concatenated.category=='Other')]

,text,category


In [63]:
pd.reset_option('display.max_colwidth')

In [64]:
df_unseen_concatenated.to_excel('labelled_unseen_data.xlsx')